In [1]:
import pandas as pd
import requests
import re
import numpy as np
from datetime import datetime

In [2]:
conflict_df = pd.read_csv('datasets/ucdp-prio-acd-171.csv')
countries_codes = pd.read_csv('datasets/countries_codes_and_coordinates.csv')

In [3]:
conflict_df;


In [4]:
for index, row in conflict_df.iterrows():
    conflict_df.loc[index, 'location'] = re.sub(r'\([^)]*\)', '', conflict_df.loc[index, 'location'])
    conflict_df.loc[index, 'location'] = conflict_df.loc[index, 'location'].rstrip()
    conflict_df.loc[index, 'sidea'] = conflict_df.loc[index, 'sidea'].replace('Government of ', '')
    conflict_df.loc[index, 'startdate'] = datetime.strptime(conflict_df.loc[index, 'startdate'], '%Y-%m-%d').month
    #conflict_df.loc[index, 'ependdate'] = datetime.strptime(conflict_df.loc[index, 'ependdate'], '%Y-%m-%d').month

    if pd.isnull(row['terr']) :
        conflict_df.loc[index, 'terr'] = row['location']    

In [6]:
conflict_df = conflict_df.drop(['gwnoa2nd', 'gwnob2nd', 'gwnoloc', 'gwnoa', 
                  'gwnob', 'version', 'startdate2', 'startprec', 
                  'startdate2', 'startprec2' , 'ependprec', 'sidea2nd', 
                  'sideb2nd', 'epend'], axis=1)

In [8]:
conflict_df = conflict_df.rename(index=str, columns={ "startdate" : "start_month", "ependdate": "end_month"});


In [9]:
conflict_df

,conflictid,location,sidea,side b,sidebid,incomp,terr,year,intensity,cumint,type,start_month,end_month,region
0,200,Bolivia,Bolivia,Popular Revolutionary Movement,719,2,Bolivia,1946,2,1,3,7,1946-07-21,5
1,200,Bolivia,Bolivia,MNR,720,2,Bolivia,1952,1,1,3,7,1952-04-12,5
2,200,Bolivia,Bolivia,ELN,721,2,Bolivia,1967,1,1,3,7,1967-10-16,5
3,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1946,1,0,1,8,NaN,3
4,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1947,1,0,1,8,NaN,3
5,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1948,1,0,1,8,NaN,3
6,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1949,1,0,1,8,NaN,3
7,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1950,1,0,1,8,NaN,3
8,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1951,1,0,1,8,NaN,3
9,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1952,1,0,1,8,NaN,3


In [10]:
iso_countries = countries_codes['Country'].tolist()
iso_code = countries_codes['ISO2'].tolist()
sidea = conflict_df['sidea'].tolist()
location  = conflict_df['location'].tolist()

In [11]:
iso_conflict = []

for i in range(0, len(sidea)):
    if sidea[i] in iso_countries or location[i] in iso_countries:
        for j in range(0, len(iso_countries)): 
            if location[i] == iso_countries[j]:
                iso_conflict.append(iso_code[j])
                break
            elif sidea[i] == iso_countries[j]:
                iso_conflict.append(iso_code[j])
                break
    else:
        iso_conflict.append(0)


In [15]:
conflict_df.head()

,conflictid,location,sidea,side b,sidebid,incomp,terr,year,intensity,cumint,type,start_month,end_month,region,ISO2
0,200,Bolivia,Bolivia,Popular Revolutionary Movement,719,2,Bolivia,1946,2,1,3,7,1946-07-21,5,BO
1,200,Bolivia,Bolivia,MNR,720,2,Bolivia,1952,1,1,3,7,1952-04-12,5,BO
2,200,Bolivia,Bolivia,ELN,721,2,Bolivia,1967,1,1,3,7,1967-10-16,5,BO
3,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1946,1,0,1,8,NaN,3,KH
4,201,Cambodia,France,Khmer Issarak,160,1,Cambodia,1947,1,0,1,8,NaN,3,KH


In [16]:
conflict_df = conflict_df.assign(ISO2 = iso_conflict )

In [17]:
conflict_df.to_csv('clean_conflict.csv')